# Deel 2: Missing Values/Proximity Matrix
Wat zijn de grote krachten van een randomforest? Dat het relatief minder uitmaakt als bij andere algoritmes dat de data schoon is. Hieronder gaan we een paar oefeningen doen om deze missende data zo goed mogelijk te interpreteren.

In [ ]:
import warnings
import operator
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

random.seed(630)
warnings.filterwarnings('ignore')

We laden hier de heart-disease dataset in, maar we verwijderen allerlei random data uit de dataset.<br>
Wij gaan proberen om die waarden zo goed mogelijk in te vullen en een zo hoog mogelijke score te krijgen.

In [ ]:
heart_df = pd.read_csv('heart.csv')
# Train data wordt gemaakt voor verder gebruik
X_mf = heart_df.drop('target', axis=1)
y_mf = heart_df[['target']]
X_mf.head()

In [ ]:
y_mf.head()

In de lijst <b>zeroedout</b> staan alle row indices en kolomnamen die uit de dataset gehaald zijn.

In [ ]:
zeroedout = []
for i in range(int(X_mf.size * 0.3)):
    row = random.randint(0, len(X_mf) - 1)
    col = random.choice((X_mf).columns)
    zeroedout.append((row, col))
zeroedout[:10]

### Laten we onze baseline zetten

Hieronder veranderen we alle zeroedout cellen naar 0.<br>
Daarna testen we welke score dit opleverd.<br>
Dit levert een score op van <b>0.75</b>

In [ ]:
rfc = RandomForestClassifier(random_state=0)
rfc

In [ ]:
for i in zeroedout:
    index = i[0] # Dit is de index van row die aangepast moet worden
    col = i[1] # Dit is de kolomnaam van de row die aangepast moet worden
    X_mf[col][index] = 0 # Je moet X_mf veranderen naar de correcte schattingswaarde (in dit geval alles naar 0)

# Hieronder wordt de data gesplit en wordt het model gefit en gescored
X_train_mf, X_test_mf, y_train_mf, y_test_mf = train_test_split(X_mf, y_mf, random_state=0)
rfc.fit(X_train_mf, y_train_mf)
rfc.score(X_test_mf, y_test_mf)

Probeer nu hetzelfde, maar verander dan alle zeroedout data naar het gemiddelde, mediaan en de modus.<br>
Laat vervolgens de score voor elke soort zien.

In [ ]:
# Mean

In [ ]:
# Mode

In [ ]:
# Median

Over het algemeen is er nog een beter manier om een eerste schatting te maken, en dat is om het gemiddelde te pakken voor numerieke datapunten en de modus voor categoriale. Maar dan niet het gemiddelde en modus van de gehele dataset, maar van de datapunten met dezelfde target waarde.<br><br>

De eerste stap is uitzoeken welke waarden categoriaal zijn. Zet deze in de lijst categorial_cols.

In [ ]:
categorial_cols = []

Vervolgens gaan we per zeroedoud datapunt kijken of het een categoriale waarde is.<br> Zo ja, pakken we de modus van alle waarden met target waarde. En zo nee, pakken we het gemiddelde van alle waarden met die target waarde.<br>
Welke score komt er dan uit het model?

In [ ]:
for i in zeroedout:
    index = i[0]
    col = i[1]
    # TODO

# RFC.apply
Nu we schattingen hebben ingevuld kunnen we de randomforest toepassen op alle data.<br>
Dit doen we met de apply functie. Deze functie geeft een lijst terug met per tree bij welke leaf nodes alle rows terecht komen.<br>
Standaard bij deze dataset kiest sklearn ervoor om 100 bomen te gebruiken. Dit zie je terug bij de shape[1] van de apply.

In [ ]:
leaves = rfc.apply(X_mf)
print(leaves.shape)
leaves

<b>Wanneer rows in dezelfde leaf node eindigen, definieren wij deze als soortgelijk.</b><br>
Het is niet de bedoeling dat we hier met de hyperparameters gaan knoeien, omdat de proximity matrix dan sub-optimaal zou kunnen werken. Als er te weinig vergelijkbare rows zijn, is er vaak sprake van overfitting. Terwijl andersom vaak een teken is van underfitting. Maar dit is geen probleem met de standaard parameters.

## Proximity Matrix
We houden in een proximity matrix bij welke rows soortgelijk zijn aan anderen<br>
Dus wanneer als voorbeeld regel 2 in dezelfde leaf node terecht komt als regel 3, Verhogen we deze cell met 1.<br>
Een proximity matrix is altijd inverted hetzelfde. de x en y as zijn hieronder allebei de row index.<br>

|   | 1 | 2 | 3 | 4 |
|---|---|---|---|---|
| 1 |   |   |   |   |
| 2 |   |   | 1 |   |
| 3 |   | 1 |   |   |
| 4 |   |   |   |   |

Optimaal zou zijn dat we maar een spiegelkant van de matrix zouden berekenen (en nog het liefst alleen de rows met missende data). Maar voor het programmeer gemak en duidelijkheid doen we ze allebei.

De opdracht is om een matrix terug te geven die aangeeft tot hoeverre elke row gelijksoortig is aan elke andere row.<br>
We doen dit door voor elke boom elke row met elke row te vergelijken en te kijken of deze in dezelfde leaf eindigt.

Een goede indicatie of de matrix klopt, is het gegeven dat elke row altijd maximaal gelijksoortig is aan zichzelf.<br> Dit betekent dat er een diagonale lijn van linksboven naar rechtsonder aanwezig moet zijn.

Ten slotte heeft de functie het argument "normalize". Wanneer deze True is, moeten alle proximities gedeeld door het aantal trees worden gedaan. Dit, zodat de hele matrix binnen de schaal van 0 (totaal verschillend) naar 1 (100% soortgelijk) valt. Om de laatste stap te doen (het toepassen van de matrix) moet de matrix eerst genormaliseerd zijn.

In [ ]:
def proximityMatrix(model, X, normalize=True):
     # Lijst met alle end nodes per row voor elke tree zoals hierboven uitgelegd.
    leaves = model.apply(X)
    
    # Het aantal rows en trees in de dataset
    n_rows = leaves.shape[0]
    n_trees = leaves.shape[1]
    
    # De proximity matrix heeft een shape van NxN waar N het aantal rows zijn.
    proximity_matrix = np.zeros([n_rows, n_rows])

    for tree in leaves.T: # Voor elke boom
        # TODO: Itereer door elke row in X, en itereer vervolgens weer door elke in X en kijk of ze 
        # allebei in dezelfde leaf node eindigen. Zo ja, increment de waarde in de proximity matrix cell dan met 1.
        pass
    return proximity_matrix

In [ ]:
plt.imshow(proximityMatrix(rfc, X_mf, normalize=True))
plt.colorbar();

## Het toepassen van de matrix
Omdat wij denken dat het toepassen van de matrix te ingewikkeld is om binnen de scope van deze workshop te houden, hebben wij ervoor gekozen om de functie die de matrix toepast alvast in te vullen.

Voor een duidelijke uitleg van wat er in deze functie gebeurt refereren we je naar de volgende zeer hulpzame [video](https://www.youtube.com/watch?v=sQ870aTKqiM).<br>
Deze functie gebruikt de matrix functie van hierboven. Als deze niet/fout is gemaakt, produceert deze functie een error.

In [ ]:
X_train_mf, X_test_mf, y_train_mf, y_test_mf = train_test_split(X_mf, y_mf, random_state=0)
rfc.fit(X_train_mf, y_train_mf)
rfc.score(X_test_mf, y_test_mf)

for redo in range(7):
    
    matrix = proximityMatrix(rfc, X_mf)
    for coord in zeroedout:

        new_value = 0
        sum_of_weights = sum(matrix[i[0]])
        weights = {}
        
        if i[1] in categorial_cols:  # categorical
            
            for value, freq in X_mf[coord[1]].value_counts(normalize=True).items():
                weight = 0
                
                for index, j in enumerate(matrix[coord[0]]):
                    if X_mf[coord[1]][index] == value:
                        weight += j
                        
                weights[value] = freq * (weight / sum_of_weights)
                
            new_value = max(weights.items(), key=operator.itemgetter(1))[0]

        else:  # numerical
        
            for index, weight in enumerate(matrix[coord[0]]):
                new_value += (weight / sum_of_weights) * X_mf[coord[1]][index]

        X_mf[coord[1]][coord[0]] = new_value

    X_train_mf, X_test_mf, y_train_mf, y_test_mf = train_test_split(X_mf, y_mf, random_state=0)
    rfc.fit(X_train_mf, y_train_mf)
    print(rfc.score(X_test_mf, y_test_mf))